In [1]:
from skimage.transform import warp_polar
import numpy as np
from scipy.ndimage import rotate
from skimage.transform import warp_polar
import matplotlib.pyplot as plt
import math


# TODO: for some reason rotated images get distorted colors, gray instead of black and white
# TODO: currently the alignment alg is not robust at all, check for bugs and improvements


order = 3


def create_synthetic_image(size=200, pattern="sine_cosine", freq=0.1):
    """Create a synthetic image with various patterns."""
    x, y = np.ogrid[:size, :size]

    if pattern == "sine_cosine":
        image = np.sin(freq * x) + np.cos(freq * y)
    elif pattern == "checkerboard":
        image = ((x // (size // 10)) % 2) ^ ((y // (size // 10)) % 2)
    elif pattern == "stripes":
        image = np.sin(freq * x)
    elif pattern == "dots":
        image = np.zeros((size, size))
        step = size // 10
        for i in range(0, size, step):
            for j in range(0, size, step):
                image[i, j] = 1
    else:
        raise ValueError(
            "Unsupported pattern type. Choose 'sine_cosine', 'checkerboard', 'stripes', or 'dots'."
        )

    image = (image - image.min()) / (image.max() - image.min())

    # Add padding to ensure the image is fully visible when rotated
    pad_size = size // 2
    padded_image = np.pad(image, pad_width=pad_size, mode="constant", constant_values=0)
    return padded_image

In [ ]:
# ACTUALLY WORKING REALLY WELL
# VERY IMPROTANT IS TO READ https://scikit-image.org/docs/stable/auto_examples/registration/plot_register_rotation.html#sphx-glr-auto-examples-registration-plot-register-rotation-py

import numpy as np
import matplotlib.pyplot as plt

from skimage import data
from skimage.registration import phase_cross_correlation
from skimage.transform import warp_polar, rotate, rescale
from skimage.util import img_as_float

radius = 705
angle = 35
image = data.retina()
image = img_as_float(image)
rotated = rotate(image, angle)
image_polar = warp_polar(image, radius=radius, channel_axis=-1)
rotated_polar = warp_polar(rotated, radius=radius, channel_axis=-1)


patterns = ["checkerboard", "dots", "sine_cosine"]
angles = np.arange(0, 360, 17.123)

size = 200
radius = (200**2 + 200**2) ** 0.5 / 2

for pattern1 in patterns:
    for angle in angles:
        img1 = create_synthetic_image(size=size, pattern=pattern1, freq=0.1)
        img2 = create_synthetic_image(size=size, pattern=pattern1, freq=0.1)
        img2 = rotate(img2, angle, order=order)

        image_polar = warp_polar(img1, radius=radius)

        rotated_polar = warp_polar(img2, radius=radius)

        shifts, error, phasediff = phase_cross_correlation(image_polar, rotated_polar, upsample_factor=1)

        print(f"Expected value for counterclockwise rotation in degrees: " f"{angle}")
        print(f"Recovered value for counterclockwise rotation: " f"{shifts[0]}")
        error = min(np.abs(angle - shifts[0]), np.abs(360 - np.abs(angle - shifts[0])))
        print("Error: ", error)